---
layout: post
title: Lists and Filtering
---

# PopCorn Hack 1: Find Students with Scores in a Range

In [ ]:
# Example of a DataFrame - more complex data structure
student_data = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emma'],
    'Score': [95, 88, 76, 92, 84],
    'Grade': ['A', 'B', 'C', 'A', 'B']
})



def find_students_in_range(df, min_score, max_score):
    # Filter the DataFrame to find students with scores in the given range
    return df[(df['Score'] >= min_score) & (df['Score'] <= max_score)]

find_students_in_range(student_data, 80, 90)



# PopCorn Hack 2: Calculate Letter Grades

In [ ]:
def add_letter_grades(df):
    def get_letter(score):
        if score >= 90:
            return 'A'
        elif score >= 80:
            return 'B'
        elif score >= 70:
            return 'C'
        elif score >= 60:
            return 'D'
        else:
            return 'F'
    
    df['Letter'] = df['Score'].apply(get_letter)
    return df

# Sample student data

student_data = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emma'],
    'Score': [95, 88, 76, 92, 84],
    'Grade': ['A', 'B', 'C', 'A', 'B']
})

# Add letter grades
add_letter_grades(student_data)



# PopCorn Hack 3: Find the Mode in a Series

In [ ]:
def find_mode(series):
    return series.mode().iloc[0]
import pandas as pd


print(find_mode(pd.Series([1, 2, 2, 3, 4, 2, 5])))


# Homework Assignment: Data Analysis

## just the prediction model for reference

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

class ParkingAvailabilityModel:
    def __init__(self):
        df = pd.read_csv('data/datasets/treas_parking_payments_2025_datasd.csv')

        df['date_trans_start'] = pd.to_datetime(df['date_trans_start'], errors='coerce')
        df['day_of_week'] = df['date_trans_start'].dt.dayofweek
        df['hour_of_day'] = df['date_trans_start'].dt.hour
        df['parking_available'] = 1
        df['lagged_parking_available'] = df.groupby('pole_id')['parking_available'].shift(1).fillna(0)
        df['time_slot'] = df['day_of_week'].astype(str) + '_' + df['hour_of_day'].astype(str)
        df['lagged_day_interaction'] = df['lagged_parking_available'] * df['day_of_week']
        df['trans_amt_binned'] = pd.cut(df['trans_amt'], bins=[0, 5, 10, 20, 50, 100, float('inf')], labels=False)
        df = df.drop(['date_trans_start', 'date_meter_expire'], axis=1)

        pole_id_counts = df['pole_id'].value_counts()
        filtered_df = df[~df['pole_id'].isin(pole_id_counts[pole_id_counts == 1].index)]
        filtered_df.loc[:, 'parking_available'] = np.where((filtered_df['hour_of_day'] >= 7) & (filtered_df['hour_of_day'] <= 19), 1, 0)

        X = filtered_df[['pole_id', 'day_of_week', 'hour_of_day', 'lagged_parking_available',
                         'time_slot', 'lagged_day_interaction', 'trans_amt_binned']]
        y = filtered_df['parking_available']

        X.loc[:, 'pole_id'] = X['pole_id'].astype('category').cat.codes

        self.imputer = SimpleImputer(strategy='most_frequent')
        X_train_imputed = self.imputer.fit_transform(X)

        self.model = LogisticRegression(max_iter=1000, solver='liblinear')
        self.model.fit(X_train_imputed, y)

    def predict(self, pole_id, day_of_week, hour_of_day):
        input_data = pd.DataFrame([[pole_id, day_of_week, hour_of_day, 0, f"{day_of_week}_{hour_of_day}", 0, 0]],
                                  columns=['pole_id', 'day_of_week', 'hour_of_day',
                                           'lagged_parking_available', 'time_slot',
                                           'lagged_day_interaction', 'trans_amt_binned'])
        input_data.loc[:, 'pole_id'] = input_data['pole_id'].astype('category').cat.codes
        input_data_imputed = self.imputer.transform(input_data)
        prob = self.model.predict_proba(input_data_imputed)[0][1]
        return round(prob * 100, 2)


## Data Loading

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('data/datasets/treas_parking_payments_2025_datasd.csv')

df['date_trans_start'] = pd.to_datetime(df['date_trans_start'], errors='coerce')
df['day_of_week'] = df['date_trans_start'].dt.day_name()
df['hour'] = df['date_trans_start'].dt.hour

df = df.dropna(subset=['date_trans_start', 'trans_amt'])

# Show the first few rows
df.head()


## Algorithms

###  Find parking transactions with highest and lowest transaction amounts

In [ ]:

# Pandas 
highest = df[df['trans_amt'] == max_amt]
lowest = df[df['trans_amt'] == min_amt]

highest[['pole_id', 'trans_amt', 'date_trans_start']]
lowest[['pole_id', 'trans_amt', 'date_trans_start']]


### Calculate the difference between max and min transaction amount per pole

In [ ]:
pole_diff = df.groupby('pole_id')['trans_amt'].agg(['max', 'min'])
pole_diff['difference'] = pole_diff['max'] - pole_diff['min']
pole_diff.head()

### Identify all poles where a transaction exceeded the average amount

In [ ]:
avg_trans_amt = df['trans_amt'].mean()
above_avg = df[df['trans_amt'] > avg_trans_amt]

print(f"Average transaction amount: ${avg_trans_amt:.2f}")
above_avg[['pole_id', 'trans_amt', 'date_trans_start']].head()

### Group by day of week and hour, and calculate average transaction

In [ ]:
grouped = df.groupby(['day_of_week', 'hour'])[['trans_amt']].mean().reset_index()
grouped.head()

# ANALYSIS + questions

### Transactions vs day

In [ ]:
day_avg = df.groupby('day_of_week')['trans_amt'].mean().sort_values()
day_avg.plot(kind='bar', title="Average Transaction by Day", figsize=(8, 4))

This shows which days have higher transaction amounts, and also indicates more parking demand.

### Which hour of the day has the highest average transaction amount?

In [ ]:
hour_avg = df.groupby('hour')['trans_amt'].mean().sort_values(ascending=False)
top_hour = hour_avg.idxmax()
top_value = hour_avg.max()
print(f"Highest average transaction is at {top_hour}:00 with ${top_value:.2f}")

### What percentage of transactions were over $5.00?

In [ ]:
over_5 = df[df['trans_amt'] > 5]
percent_over_5 = (len(over_5) / len(df)) * 100
print(f"{percent_over_5:.2f}% of transactions were over $5.00")


# SQLite Db

In [ ]:
from __init__ import db, app
from model.parking import ParkingSpot
from model.user import User

class ParkingData(db.Model):
    __tablename__ = 'parking_data'

    id = db.Column(db.Integer, primary_key=True)
    _amount_paid = db.Column(db.Float, nullable=False)
    _user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    _parking_spot_id = db.Column(db.Integer, db.ForeignKey('parking_spots.id'), nullable=False)
    _transaction_time = db.Column(db.DateTime, nullable=False)

    def __init__(self, amount_paid, user_id, parking_spot_id, transaction_time):
        self._amount_paid = amount_paid
        self._user_id = user_id
        self._parking_spot_id = parking_spot_id
        self._transaction_time = transaction_time

    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
        except Exception as e:
            db.session.rollback()
            raise e


## Some queries

In [ ]:
query = db.session.query(
    ParkingData._parking_spot_id,
    db.func.avg(ParkingData._amount_paid).label('avg_amount_paid')
).group_by(ParkingData._parking_spot_id)

df_avg_per_spot = pd.read_sql(query.statement, db.session.bind)
print(df_avg_per_spot)


In [ ]:
query = db.session.query(
    ParkingData._parking_spot_id,
    db.func.max(ParkingData._amount_paid).label('max_amount_paid'),
    db.func.min(ParkingData._amount_paid).label('min_amount_paid')
).group_by(ParkingData._parking_spot_id)

df_high_low = pd.read_sql(query.statement, db.session.bind)
print(df_high_low)


In [ ]:
query = db.session.query(
    ParkingData._user_id,
    db.func.count().label('transaction_count')
).group_by(ParkingData._user_id)

df_user_transactions = pd.read_sql(query.statement, db.session.bind)
print(df_user_transactions)


## Analysis/ Comparison

Pandas is helpful when you want to quickly look at and work with data in Python, especially if the file is small and on your computer. SQL is better when the data is large or stored in a database, because it can filter and sort before loading. Pandas is easier for testing ideas, but SQL is more efficient for handling big or shared data.

